In [ ]:
import pandas as pd
from functools import reduce

pd.set_option('display.max_colwidth', -1)

tw_df = pd.read_csv('influencers-project/data/twitter_influencers_details.csv', sep=';')
ig_df = pd.read_csv('influencers-project/data/instagram_influencers_details.csv', sep=';')
yt_df = pd.read_csv('influencers-project/data/youtube_influencers_details.csv', sep=';')
kl_df = pd.read_csv('influencers-project/data/klout_influencers_details.csv', sep=';')
fb_df = pd.read_csv('influencers-project/data/facebook_influencers_details.csv', sep=';')

# print(tw_df.shape)
# print(ig_df.shape)
# print(yt_df.shape)
# print(kl_df.shape)


dfs = [fb_df, ig_df, yt_df, kl_df]
df_final = reduce(lambda left,right: pd.merge(left,right,on='tw_handle', how='right'), dfs)
# print(df_final.shape)
# print(df_final.sample(5))


df_final.to_csv('all_influencers_details.csv', sep=';', index=False)

In [ ]:
import pandas as pd
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.influencers_db
collection = db.combined_collection

df = pd.read_csv('all_influencers_details.csv', sep=';')
records = df.to_dict(orient='records')

result = collection.insert_many(records)

In [ ]:
db.test_coll.find_one({'tw_handle': 'MKBHD'})

In [ ]:
import json
from os import listdir
from os.path import isfile, join

from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.test_database
collection = db.twitter_collection

path = '/Users/manojkarthick/Documents/Spring-18/CMPT-733/Project/twitter-data/'
twitter_files = [f for f in listdir(path) if isfile(join(path, f))]
twitter_files.remove('.DS_Store')

for twitter_file in twitter_files:
    file_name = join(path, twitter_file)
    fp = open(file_name, 'r')
    twitter_data = [line for line in fp.readlines()]
    for data in twitter_data:
        json_data = json.loads(data)
        collection.insert_one(json_data)

In [ ]:
import json
from os import listdir
from os.path import isfile, join
from pymongo import MongoClient

client = MongoClient('localhost', 27017)

def dump_data(path):    
    files = [f for f in listdir(path) if isfile(join(path, f))]
    if '.DS_Store' in files:
        files.remove('.DS_Store')

    for f in files:
        file_name = join(path, f)
        fp = open(file_name, 'r')
        twitter_data = [line for line in fp.readlines()]
        for data in twitter_data:
            json_data = json.loads(data)
            collection.insert_one(json_data)

db = client.influencers_db
collection = db.instagram_collection
path = '/Users/manojkarthick/Documents/Spring-18/CMPT-733/Project/instagram-data/'

dump_data(path)

In [12]:
# Generate needed CSV
import csv
import json
from os import listdir
from os.path import isfile, join

path = 'keras-vgg-export'
opath = 'keras-vgg-export/reformatted/'
export_files = [f for f in listdir(path) if isfile(join(path, f)) and (not f.startswith('.'))]

for export_file in export_files:
    with open(join(opath,export_file), 'w') as f:
        writer = csv.writer(f)
        writer.writerow(('class_name', 'class_description', 'score', 'image', 'model'))
        for line in open(join(path, export_file), 'r'):
            json_array = json.loads(line)
            for json_value in json_array:
                class_name = json_value['class_name']
                class_description = json_value['class_description']
                score = json_value['score']
                image = json_value['image']
                model = json_value['model']
                row = (class_name, class_description, score, image, model)
                writer.writerow(row)

In [ ]:
from pymongo import MongoClient
from os import listdir, makedirs
from os.path import isfile, join, exists
import pandas as pd


# client = MongoClient('localhost', 27017)
# db = client.influencers_db
# collection = db.cv_collection

path = '/Users/manojkarthick/Documents/Spring-18/CMPT-733/Project-733/fb-comments-data/'

export_files = [f for f in listdir(path) if isfile(join(path, f)) and (not f.startswith('.'))]

for export_file in export_files:
    fname = join(path, export_file)
    df = pd.read_json(fname, orient='records')
    print(df.head())
    break

In [ ]:
from pymongo import MongoClient
from os import listdir, makedirs
from os.path import isfile, join, exists
import pandas as pd

client = MongoClient('localhost', 27017)
db = client.influencers_db
collection = db.twitter_new_collection

path = '/Users/manojkarthick/Documents/Spring-18/CMPT-733/Project-733/twitter-data-new/'

export_files = [f for f in listdir(path) if isfile(join(path, f)) and (not f.startswith('.'))]

for export_file in export_files:
    fname = join(path, export_file)
    df = pd.read_json(fname, lines=True)
    records = df.to_dict(orient='records')
    op = collection.insert_many(records)
    print(op)

In [16]:
path = '/Users/manojkarthick/Documents/Spring-18/CMPT-733/Project-733/insta_followers_data.csv'
igdf = pd.read_csv(path)

collection = db.instagram_followers_collection

records = igdf.to_dict(orient='records')
op = collection.insert_many(records)
print(op)

In [24]:
from pymongo import MongoClient
from os import listdir, makedirs
from os.path import isfile, join, exists
import pandas as pd
import json

client = MongoClient('localhost', 27017)
db = client.influencers_db
collection = db.facebook_new_collection

path = '/Users/manojkarthick/Documents/Spring-18/CMPT-733/Project-733/data_updated_comments_fb/'

export_files = [f for f in listdir(path) if isfile(join(path, f)) and (not f.startswith('.'))]

for export_file in export_files:
    if export_file == 'MKBHD':
        fname = join(path, export_file)
        dataf = open(fname, 'r')
        data=dataf.read().splitlines()
        dataf.close()
        js=[]

        for d in data:
            js.append(json.loads(d))
        df=pd.DataFrame(js)

    #     print(fname)
    #     df = pd.read_json(fname, lines=True)
        records = df.to_dict(orient='records')
        op = collection.insert_many(records)
        print(op)

In [33]:
from os.path import isdir
from shutil import copyfile

img_path = '/Users/manojkarthick/Documents/Spring-18/CMPT-733/Project-733/Web/flask-app/static/instagram-images'
img_dirs = [f for f in listdir(img_path) if isdir(join(img_path, f)) and (not f.startswith('.'))]

for img_dir in img_dirs:
    abs_img_dir = join(img_path,img_dir)
    img_files = [f for f in listdir(abs_img_dir) if isfile(join(abs_img_dir, f)) and (not f.startswith('.'))]
    dest_dir = '/Users/manojkarthick/Documents/Spring-18/CMPT-733/Project-733/Web/flask-app/static/instagram-images-less/{}/'.format(img_dir)
    makedirs(dest_dir)
    count = 0
    print("Copying: {}".format(img_dir))
    for img_file in img_files:
        if count == 50:
            break
        src = join(abs_img_dir, img_file)
        dst = dest_dir + img_file
        copyfile(src, dst)

        count += 1

Copying: abeautifulmess
Copying: adventurouskate
Copying: aimeesong
Copying: AndroidAuth
Copying: austinnotduncan
Copying: BabyAriel
Copying: beautylish
Copying: blogilates
Copying: brightbazaar
Copying: bryanboy
Copying: camerondallas
Copying: chiaraferragni
Copying: chrisburkard
Copying: christendtweets
Copying: cococozy
Copying: copycatchic
Copying: CupcakeAddictAU
Copying: damonandjo
Copying: dantdm
Copying: Dave2Dtv
Copying: designlovefest
Copying: designsponge
Copying: em_henderson
Copying: fitmencook
Copying: funforlouis
Copying: gabifresh
Copying: GalMeetsGlam
Copying: GuyKawasaki
Copying: harto
Copying: hudabeauty
Copying: iisuperwomanii
Copying: ijustine
Copying: imjennim
Copying: inspiralized
Copying: izyhossack
Copying: Jack_Septic_Eye
Copying: JeffreeStar
Copying: jenjentrixie
Copying: JenSelter
Copying: JohnnyJet
Copying: joythebaker
Copying: kandeejohnson
Copying: kayla_itsines
Copying: kingbach
Copying: ladolcevitablog
Copying: lelepons
Copying: LinusTech
Copying: logan